In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
! mkdir ~/.kaggle

In [ ]:
!cp /content/drive/MyDrive/kaggle/kaggle.json ~/.kaggle/kaggle.json

In [ ]:
! chmod 600 ~/.kaggle/kaggle.json

In [ ]:
! kaggle competitions download -c dog-breed-identification

In [ ]:
! unzip dog-breed-identification.zip

In [2]:
import pandas as pd
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications.inception_resnet_v2 import InceptionResNetV2
from tensorflow.keras.applications import InceptionV3
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D, Dropout
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping

In [ ]:
base_dir = 'train'
img_size = (400, 400)
batch_size = 32

labels = pd.read_csv('labels.csv')
labels['id'] = labels['id'].apply(lambda x: x +'.jpg')

train_datagen = ImageDataGenerator(rescale=1./255,
                                   rotation_range=20,
                                   width_shift_range=0.2,
                                   height_shift_range=0.2,
                                   shear_range=0.2,
                                   horizontal_flip=True,
                                   fill_mode='nearest',
                                   validation_split=0.2)

train_generator = train_datagen.flow_from_dataframe(dataframe=labels,
                                                    directory=base_dir,
                                                    x_col='id',
                                                    y_col='breed',
                                                    target_size=img_size,
                                                    batch_size=batch_size,
                                                    class_mode='categorical',
                                                    subset='training')

validation_generator = train_datagen.flow_from_dataframe(dataframe=labels,
                                                         directory=base_dir,
                                                         x_col='id',
                                                         y_col='breed',
                                                         target_size=img_size,
                                                         batch_size=batch_size,
                                                         class_mode='categorical',
                                                         subset='validation')

Found 8178 validated image filenames belonging to 120 classes.
Found 2044 validated image filenames belonging to 120 classes.


In [ ]:
def setup_inceptionresnetv2_model(num_classes):
    base_model = InceptionResNetV2(weights='imagenet', include_top=False)
    x = base_model.output
    x = Dropout(0.3)(x)
    x = GlobalAveragePooling2D()(x)
    predictions = Dense(num_classes, activation='softmax')(x)
    model = Model(inputs=base_model.input, outputs=predictions)

    for layer in base_model.layers:
        layer.trainable = False

    early_stopping = EarlyStopping(monitor='val_loss', patience=3)
    model.compile(optimizer=Adam(learning_rate=0.001), loss='categorical_crossentropy', metrics=['accuracy'])
    return model

model_inceptionresnetv2 = setup_inceptionresnetv2_model(len(train_generator.class_indices))
early_stopping = EarlyStopping(monitor='val_loss', patience=3)
history = model_inceptionresnetv2.fit(train_generator, validation_data=validation_generator, epochs=10, callbacks=[early_stopping])

219055592/219055592 [==============================] - 6s 0us/step
Epoch 1/10
256/256 [==============================] - 628s 2s/step - loss: 1.6297 - accuracy: 0.7838 - val_loss: 0.4502 - val_accuracy: 0.9139
Epoch 2/10
256/256 [==============================] - 568s 2s/step - loss: 0.3460 - accuracy: 0.9149 - val_loss: 0.3296 - val_accuracy: 0.9159
Epoch 3/10
256/256 [==============================] - 544s 2s/step - loss: 0.2734 - accuracy: 0.9228 - val_loss: 0.3043 - val_accuracy: 0.9119
Epoch 4/10
256/256 [==============================] - 531s 2s/step - loss: 0.2446 - accuracy: 0.9270 - val_loss: 0.2812 - val_accuracy: 0.9198
Epoch 5/10
256/256 [==============================] - 575s 2s/step - loss: 0.2231 - accuracy: 0.9290 - val_loss: 0.2892 - val_accuracy: 0.9159
Epoch 6/10
256/256 [==============================] - 547s 2s/step - loss: 0.2113 - accuracy: 0.9318 - val_loss: 0.2633 - val_accuracy: 0.9227
Epoch 7/10
256/256 [==============================] - 600s 2s/step - loss: 

In [ ]:
def setup_inceptionv3_model(num_classes):
    base_model = InceptionV3(weights='imagenet', include_top=False)
    x = base_model.output
    x = Dropout(0.3)(x)
    x = GlobalAveragePooling2D()(x)
    predictions = Dense(num_classes, activation='softmax')(x)
    model = Model(inputs=base_model.input, outputs=predictions)

    for layer in base_model.layers:
        layer.trainable = False

    early_stopping = EarlyStopping(monitor='val_loss', patience=3)
    model.compile(optimizer=Adam(learning_rate=0.001), loss='categorical_crossentropy', metrics=['accuracy'])
    return model

model_inceptionv3 = setup_inceptionv3_model(len(train_generator.class_indices))
early_stopping = EarlyStopping(monitor='val_loss', patience=3)
history = model_inceptionv3.fit(train_generator, validation_data=validation_generator, epochs=10, callbacks=[early_stopping])

87910968/87910968 [==============================] - 5s 0us/step
Epoch 1/10
256/256 [==============================] - 496s 2s/step - loss: 1.8271 - accuracy: 0.7011 - val_loss: 0.6329 - val_accuracy: 0.8591
Epoch 2/10
256/256 [==============================] - 464s 2s/step - loss: 0.4537 - accuracy: 0.8935 - val_loss: 0.4250 - val_accuracy: 0.8870
Epoch 3/10
256/256 [==============================] - 473s 2s/step - loss: 0.3304 - accuracy: 0.9137 - val_loss: 0.3760 - val_accuracy: 0.8855
Epoch 4/10
256/256 [==============================] - 468s 2s/step - loss: 0.2847 - accuracy: 0.9170 - val_loss: 0.3864 - val_accuracy: 0.8787
Epoch 5/10
256/256 [==============================] - 470s 2s/step - loss: 0.2548 - accuracy: 0.9230 - val_loss: 0.3374 - val_accuracy: 0.8977
Epoch 6/10
256/256 [==============================] - 467s 2s/step - loss: 0.2254 - accuracy: 0.9299 - val_loss: 0.3474 - val_accuracy: 0.8924
Epoch 7/10
256/256 [==============================] - 473s 2s/step - loss: 0.